In [1]:
from pathlib import Path
import pandas as pd
from alphalens.tears import create_summary_tear_sheet
from alphalens.utils import get_clean_factor_and_forward_returns

In [2]:
# GLOBAL SETTINGS

# Directories
model_dataDir = 'DATA/MODELDATA/'
model_plot_dataDir = 'DATA/MODELRESULTS/'

# Time
START = 2020
END = 2022



#Helpers
idx = pd.IndexSlice



In [3]:
# Helper Functions
'''
def get_trade_prices(tickers, start, stop):
    prices = model_mdf
    prices = prices.loc[idx[tickers, str(start):str(stop)], 'Open']
    return (prices
            .unstack('Coin')
            .sort_index()
            .shift(-1)
            .tz_localize('UTC'))
'''

def get_trade_prices(start, stop):
    model_mdf = pd.read_csv(model_dataDir + 'ModelData.csv')
    model_mdf.rename(columns={'Unnamed: 0': 'Dates'}, inplace=True)
    model_mdf['Dates'] = pd.to_datetime(model_mdf['Dates'])
    model_mdf.set_index(['Dates', 'Coin'], inplace=True)
    model_mdf = model_mdf.loc[:,'Open']
    #model_mdf = model_mdf.loc[idx[tickers, str(START):str(END)], 'Open']
    prices = model_mdf.unstack('Coin').sort_index().shift(-1).tz_localize('UTC')
    
    return prices



def get_best_alpha(scores):
    return scores.groupby('alpha').ic.mean().idxmax()


def get_factor(predictions):
    return (predictions.unstack('Coin')
            .dropna(how='all')
            .stack()
            .tz_localize('UTC', level='Dates')
            .sort_index())    

In [4]:
# Load the Model Results Data
lr_predictions = pd.read_hdf(f'{model_dataDir}lr_model_predictions.h5', 'lr/predictions')
ridge_scores = pd.read_hdf(f'{model_dataDir}Ridge_Scores.h5', 'ridge/scores')
ridge_predictions = pd.read_hdf(f'{model_dataDir}Ridge_Predictions.h5', 'ridge/predictions')
lasso_scores = pd.read_hdf(f'{model_dataDir}Lasso_Scores.h5', 'lasso/scores')
lasso_predictions = pd.read_hdf(f'{model_dataDir}Lasso_Predictions.h5', 'lasso/predictions')

# Load the OHLV Price data
#model_mdf = pd.read_csv(model_dataDir + 'ModelData.csv')
#model_mdf.rename(columns={'Unnamed: 0': 'Dates'}, inplace=True)
#model_mdf['Dates'] = pd.to_datetime(model_mdf['Dates'])
#model_mdf.set_index(['Dates', 'Coin'], inplace=True)
#model_mdf = model_mdf.loc[:,'Open':'Volume']
#prices = model_mdf.unstack('Coin').sort_index().shift(-1).tz_localize('UTC')

In [5]:
# Now we will check the Alpha and Beta metrics of the signals found by our models in the prior step

In [6]:
# From Linear Regression
lr_factor = get_factor(lr_predictions.predicted.swaplevel())
lr_factor.head()

Dates                      Coin                 
2020-02-01 00:00:00+00:00  algorand                 0.025134
                           arweave                  0.016334
                           basic-attention-token    0.016569
                           binancecoin             -0.008778
                           bitcoin                 -0.004959
dtype: float64

In [7]:
#tickers = lr_factor.index.get_level_values('Coin').unique()

In [8]:
trade_prices = get_trade_prices(2020, 2022)
trade_prices.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 745 entries, 2020-01-01 00:00:00+00:00 to 2022-01-14 00:00:00+00:00
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   algorand               744 non-null    float64
 1   arweave                744 non-null    float64
 2   basic-attention-token  744 non-null    float64
 3   binancecoin            744 non-null    float64
 4   bitcoin                744 non-null    float64
 5   bitcoin-cash           744 non-null    float64
 6   bitcoin-cash-sv        744 non-null    float64
 7   bittorrent-2           744 non-null    float64
 8   cardano                744 non-null    float64
 9   chainlink              744 non-null    float64
 10  cosmos                 744 non-null    float64
 11  crypto-com-chain       744 non-null    float64
 12  dash                   744 non-null    float64
 13  decentraland           744 non-null    float64
 14  dogecoin 

In [9]:
print(len(trade_prices.columns))

41


In [10]:
trade_prices.head()

Coin,algorand,arweave,basic-attention-token,binancecoin,bitcoin,bitcoin-cash,bitcoin-cash-sv,bittorrent-2,cardano,chainlink,...,monero,okb,ripple,stellar,terra-luna,tezos,theta-token,tron,vechain,zcash
Dates,,,,,,,,,,,,,,,,,,,,,
2020-01-01 00:00:00+00:00,0.217110,0.652953,0.190072,13.522027,7139.044338,201.799270,95.251893,0.000280,0.032875,1.784257,...,45.668221,2.583228,0.191127,0.044894,0.217308,1.351242,0.085440,0.013090,0.005313,27.657228
2020-01-02 00:00:00+00:00,0.225794,0.698436,0.185653,13.579740,7196.883716,203.727639,96.466791,0.000283,0.033663,1.780386,...,48.339400,2.612412,0.190122,0.045167,0.204538,1.272692,0.085352,0.013154,0.005259,27.393856
2020-01-03 00:00:00+00:00,0.228031,0.668637,0.186281,13.650140,7320.309525,221.811173,104.624500,0.000289,0.033864,1.791668,...,50.422485,2.619287,0.192036,0.045401,0.231951,1.277193,0.088620,0.013391,0.005350,28.276526
2020-01-04 00:00:00+00:00,0.234223,0.597861,0.190017,13.972162,7448.688305,223.863172,110.612056,0.000290,0.034543,1.804323,...,51.035368,2.678726,0.194730,0.045771,0.239527,1.268082,0.092812,0.013656,0.005580,30.396363
2020-01-05 00:00:00+00:00,0.233386,0.722573,0.191317,14.380112,7520.122401,236.732977,115.716651,0.000296,0.035271,1.838128,...,55.979012,2.711035,0.196440,0.046401,0.234815,1.268031,0.099747,0.013867,0.005519,31.452828
